In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
import pandas as pd
import numpy as np
import re
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

# DATA_PATH = "/content/drive/MyDrive/생성 AI 모델링/data/"
DATA_PATH = "/content/drive/MyDrive/멀티캠퍼스 자료/Machine Learning/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device


'cpu'

# 제로샷 Classification

In [4]:
!pip install transformers

In [136]:
train_ft = pd.read_csv(f'{DATA_PATH}train_ft_38000_0918.csv')

In [137]:
train_ft

,review,reply,sentiment,score,kiwi_reviews,category
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,1,0.566494,"['맛있', '먹', '묵사발', '시원', '맛있']",['맛']
1,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,고객님께서도 행복하시길 바랄게요,1,0.608408,"['양', '푸짐', '맛있', '청국장', '냄새', '나', '콩', '들', ...","['양', '맛']"
2,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,고객님의 행복한 시간에 저희 매장이 함께 할 수 있어 너무나도 영광입니다! 즐거움이...,1,0.381793,"['냉면', '시키', '미니', '냉면', '합치', '양', '적', '보이',...",['양']
3,맛나게 잘 묵었습니다,"고객님의 즐거운 순간을 함께할 수 있기를 바라며, 기다리고 있겠습니다 더욱 발전된 ...",1,0.549870,"['맛나', '묵']",['맛']
4,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,"안녕하세요! 주문해 주셔서 진심으로 감사드려요! 이렇게 좋은 리뷰 써 주시니, 메뉴...",1,0.778149,"['맛', '양', '만족', '배달', '보내', '만족']","['양', '맛', '배달']"
...,...,...,...,...,...,...
38550,항상 맛있게 먹었는데 이번에는 음식이 식어서 별로였어요. 아쉬워요,고객님의 의견을 반영하여 다양한 맛과 양을 제공하도록 노력하겠습니다.,0,0.811507,"['맛있', '먹', '이번', '음식', '식']",['맛']
38551,언제나 맛있게 먹었는데 이번에는 음식이 지루하고 별로였어요. 아쉬워요,"닭도리탕의 맛이 매우 좋지 않고, 김치가 물렸다는 점 죄송합니다. 앞으로 더 맛있는...",0,0.827452,"['맛있', '먹', '이번', '음식', '지루하']",['맛']
38552,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...",고객님. 말씀해주신 점 먼저 진심으로 사과드립니다. 맛과 촉감에 실망하셨다니 정말 ...,0,0.942769,"['맛', '음식', '촉촉', '식', '먹', '그렇', '다음', '시키', ...",['맛']
38553,"맛도 없고, 배달도 늦어서 너무 실망했습니다",고객님께 실망을 드려 죄송합니다. 저희 음식이 맛있지 않아서 큰 실망을 받으셨군요....,0,0.912723,"['맛', '없', '배달', '늦', '실망']","['맛', '배달']"


# 답변 생성 (키워드 + 유사도)

In [5]:
# AutoModelForCausalLM 인과 관련 사용
from transformers import AutoTokenizer, AutoModelForCausalLM

In [12]:
model_name = "skt/kogpt2-base-v2"

In [13]:
# AutoModelForCausalLM 인과 관련 사용
from transformers import AutoTokenizer, AutoModelForCausalLM

In [14]:
model = AutoModelForCausalLM.from_pretrained(model_name)

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          bos_token='</s>',
                                          eos_token='</s>',
                                          unk_token='<unk>',
                                          pad_token='<pad>',
                                          mask_token='<mask>',
                                          max_len=1024)

In [30]:
loaded_model = AutoModelForCausalLM.from_pretrained(f"{DATA_PATH}0908_gpt_freeze_all").to(device)

In [ ]:
# neg_model = AutoModelForCausalLM.from_pretrained(f"{DATA_PATH}0908_negative_kogpt2_chat").to(device)
# # pos_model = AutoModelForCausalLM.from_pretrained(f"{DATA_PATH}0908_positive_kogpt2_chat").to(device)

## 테스트루프

In [ ]:
# @torch.no_grad()
# def chatbot(model, tokenizer, max_len, device, num_samples=3):
#     model.eval()
#     while True:
#         text = input("user > ").strip()
#         if text == "quit":
#             break

#         text = "<q>" + text + "</s><a>"
#         x = tokenizer.encode(text, return_tensors="pt").to(device)
#         q_len = len(text) + 1

#         generated_texts = [] # 답변 후보군을 담을 리스트
#         for _ in range(num_samples):
#             result_ids = model.generate(x,
#                                         max_length=max_len,
#                                         repetition_penalty=2.0,
#                                         use_cache=True,
#                                         do_sample=True,
#                                         temperature=0.9,
#                                         top_k=50,
#                                         num_return_sequences=1
#                                         )
#             generated_text = tokenizer.decode(result_ids[0])
#             generated_text = generated_text[q_len:-4]
#             generated_texts.append(generated_text)

#         print("bot >")
#         for i, generated_text in enumerate(generated_texts, 1):
#             print(f"{i}. {generated_text}")
#         print("--------------------------------------------------------------------------------------------")

In [7]:
!pip install -U sentence-transformers

In [33]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.feature_extraction.text import CountVectorizer

bert_model = SentenceTransformer('jhgan/ko-sroberta-multitask')

def jaccard_similarity(text1, text2):
    # 문장을 단어로 분할 (또는 집합으로 변환)
    words1 = set(text1.split())
    words2 = set(text2.split())

    # 자카드 유사도 계산
    intersection = len(words1.intersection(words2))
    union = len(words1) + len(words2) - intersection

    if union == 0:
        return 0.0
    else:
        return float(intersection) / union

def evaluate_similarity(input_text, generated_text):
    input_embedding = bert_model.encode(input_text)
    generated_embedding = bert_model.encode(generated_text)

    # BERT 임베딩 기반 코사인 유사도
    cosine_similarity = 1 - pairwise_distances([input_embedding], [generated_embedding], metric='cosine')[0][0]

    # 자카드 유사도
    jaccard_sim = jaccard_similarity(input_text, generated_text)

    # 코사인 유사도와 자카드 유사도를 5:5 비중으로 조합
    combined_similarity = 0.5 * cosine_similarity + 0.5 * jaccard_sim

    return combined_similarity



In [35]:

# 예제
input_text = "음식이 맛있어요. 배달은 빨랐는데 양이 조금 적은 건 아쉽네요 "
generated_text = "맛있는 저희 매장 음식을 먹어주셔서 감사합니다. 배달이 느려서 죄송합니다. "
combined_sim = evaluate_similarity(input_text, generated_text)

print("Combined Similarity:", combined_sim)

Combined Similarity: 0.3371405601501465


In [ ]:

# 예제
input_text = "이 음식은 맛있다."
generated_text = "맛있는 음식을 먹어주셔서 감사합니다."
combined_sim = evaluate_similarity(input_text, generated_text)

print("Combined Similarity:", combined_sim)

In [8]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import pairwise_distances
bert_model = SentenceTransformer('jhgan/ko-sroberta-multitask')

def evaluate_similarity(input_text, generated_text):
    input_embedding = bert_model.encode(input_text)
    generated_embedding = bert_model.encode(generated_text)

    similarity_score = 1 - pairwise_distances([input_embedding], [generated_embedding], metric='cosine')[0][0] # cosine
    # similarity_score = pairwise_distances([input_embedding], [generated_embedding], metric='euclidean')[0][0] # euclidean
    # similarity_score = pairwise_distances([input_embedding], [generated_embedding], metric='manhattan')[0][0] # manhattan

    return similarity_score

In [9]:
# 옵션
from transformers import pipeline
pipe = pipeline("text2text-generation", model="lcw99/t5-base-korean-paraphrase")

def paraphrase_text(text):
    generated_text = pipe(text, max_length=512, early_stopping=True,
                          num_beams=4,
                          num_return_sequences=2,
                          no_repeat_ngram_size=4,
                          top_k=50,
                          top_p=0.95)
    generated_text = generated_text[0]['generated_text']
    return generated_text

In [38]:
from transformers import pipeline
import pandas as pd

def extract_scores(train_review, candidate_labels):
    # Zero-shot classification 모델 불러오기
    # classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")
    classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", batch_size=4)

    # 결과를 저장할 리스트 초기화
    output_scores = {label: [] for label in candidate_labels}

    # 각 리뷰에 대해 카테고리 분류 결과 추출
    for review in train_review:
        output = classifier(review, candidate_labels, multi_label=False)
        for label, score in zip(output['labels'], output['scores']):
            output_scores[label].append(score)

    # 카테고리별 점수를 DataFrame으로 변환
    df = pd.DataFrame(output_scores)
    df_review = pd.DataFrame(train_review)

    df = pd.concat([df_review,df],axis=1)

    return df


In [50]:
candidate_labels = ["맛", '양', "배달", "가격", '서비스']


In [36]:


# def chatbot_with_evaluation(model, tokenizer, max_len, device, num_samples=5):
#     model.eval()
#     while True:
#         text = input("user > ").strip()
#         if text == "quit":
#             break

#         text = "<q>" + text + "</s><a>"
#         x = tokenizer.encode(text, return_tensors="pt").to(device)
#         q_len = len(text) + 1

#         best_generated_text = None
#         best_similarity_score = -1.0
#         generated_texts = []  # 답변 후보군을 담을 리스트
#         for i in range(num_samples):
#             result_ids = model.generate(x,
#                                         max_length=max_len,
#                                         repetition_penalty=2.0,
#                                         use_cache=True,
#                                         do_sample=True,
#                                         temperature=0.9,
#                                         top_k=50,
#                                         num_return_sequences=1
#                                         )
#             generated_text = tokenizer.decode(result_ids[0])
#             generated_text = generated_text[q_len:-4]
#             generated_text = paraphrase_text(generated_text)

#             similarity_score = evaluate_similarity(text, generated_text)
#             generated_texts.append((similarity_score,generated_text))
#             print(generated_texts[i])

#             if similarity_score > best_similarity_score:
#                 best_similarity_score = similarity_score
#                 best_generated_text = generated_text

#         print()
#         print("Best_reply >", best_generated_text)
#         print("Best_similarity_score:", best_similarity_score)
#         print("--------------------------------------------------------------------------------------------")


# 라벨 확률 추출

In [138]:
from transformers import pipeline
import pandas as pd

def extract_scores(train_review, candidate_labels):
    # Zero-shot classification 모델 불러오기
    # classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")
    classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", batch_size=4)

    # 결과를 저장할 리스트 초기화
    output_scores = {label: [] for label in candidate_labels}

    # 각 리뷰에 대해 카테고리 분류 결과 추출
    for review in train_review:
        output = classifier(review, candidate_labels, multi_label=False)
        for label, score in zip(output['labels'], output['scores']):
            output_scores[label].append(score)

    # 카테고리별 점수를 DataFrame으로 변환
    df = pd.DataFrame(output_scores)
    df_review = pd.DataFrame(train_review)

    df = pd.concat([df_review,df],axis=1)

    return df


In [139]:
candidate_labels = ["맛", '양', "배달", "가격", '서비스']

extract_scores(train_ft['review'][:20],candidate_labels)

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


,review,맛,양,배달,가격,서비스
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,0.747809,0.059194,0.050575,0.016397,0.126026
1,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,0.422862,0.548768,0.004219,0.003338,0.020812
2,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,0.108987,0.254800,0.042728,0.033020,0.560466
3,맛나게 잘 묵었습니다,0.829860,0.061057,0.024354,0.009858,0.074872
4,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,0.162239,0.069010,0.482515,0.000592,0.285644
5,처음시켜보는데 맛있고 리뷰서비스도 괜찮네요. 다음에도 또시킬듯요,0.629083,0.016534,0.003376,0.002314,0.348693
6,맛있습니다! 근데 국물이 안와서 아쉽습니다.,0.916494,0.020101,0.016478,0.005886,0.041041
7,맛있게잘먹었습니다,0.817559,0.039240,0.050272,0.008535,0.084394
8,너무 맛있게 잘 먹었습니다!,0.933994,0.015302,0.008603,0.002467,0.039634
9,사진은 없지만 맛나게 잘 먹었습니다,0.888417,0.022933,0.017872,0.014717,0.056061


In [47]:
from transformers import pipeline
import pandas as pd

def extract_scores_with_target_match_evaluation(target_review, generated_text, candidate_labels):
    # Zero-shot classification 모델 불러오기
    # classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")
    classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", batch_size=4)

    # 결과를 저장할 리스트 초기화
    output_scores = {label: [] for label in candidate_labels}


    output_review = classifier(target_review, candidate_labels, multi_label=False)
    for label, score in zip(output_replies['labels'], output_replies['scores']):
        output_scores[label].append(score)

    # 각 답변에 대해 카테고리 분류 결과 추출
    for text in generated_text:
        output_replies = classifier(text, candidate_labels, multi_label=False)
        for label, score in zip(output_replies['labels'], output_replies['scores']):
            output_scores[label].append(score)

    print(generated_text,output_scores)

    # # 카테고리별 점수를 DataFrame으로 변환
    # df = pd.DataFrame(output_scores)
    # df_review = pd.DataFrame(train_review)

    # df = pd.concat([df_review,df],axis=1)

    # return df

In [65]:
from transformers import pipeline
import pandas as pd

def extract_scores_with_target_match_evaluation(target_review, generated_text, candidate_labels):
    # Zero-shot classification 모델 불러오기
    classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", batch_size=4)

    # 결과를 저장할 리스트 초기화
    output_scores_review = {label: [] for label in candidate_labels}

    # 입력된 타겟 리뷰에 대한 카테고리 분류 결과 추출
    output_review = classifier(target_review, candidate_labels, multi_label=False)
    for label, score in zip(output_review['labels'], output_review['scores']):
        output_scores_review[label].append(score)


    output_scores_reply = {label: [] for label in candidate_labels}

    # 각 생성된 텍스트에 대해 카테고리 분류 결과 추출
    for text in generated_text:
        output_replies = classifier(text, candidate_labels, multi_label=False)
        for label, score in zip(output_replies['labels'], output_replies['scores']):
            output_scores_reply[label].append(score)

    # 카테고리별 점수를 DataFrame으로 변환

    df_1 = pd.DataFrame(output_scores_review)
    df_2 = pd.DataFrame(output_scores_reply)

    # 타겟 리뷰와 생성된 텍스트를 열로 추가
    df_1['Target Review'] = target_review
    df_2['Generated Text'] = generated_text

    output_scores_review
    output_scores_reply

    return

In [68]:
from transformers import pipeline
import pandas as pd

def extract_scores_with_target_match_evaluation(target_review, generated_text, candidate_labels):
    # Zero-shot classification 모델 불러오기
    classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", batch_size=4)

    # 결과를 저장할 리스트 초기화
    output_scores_review = {label: [] for label in candidate_labels}
    output_scores_reply = {label: [] for label in candidate_labels}

    # 입력된 타겟 리뷰에 대한 카테고리 분류 결과 추출
    output_review = classifier(target_review, candidate_labels, multi_label=False)
    for label, score in zip(output_review['labels'], output_review['scores']):
        output_scores_review[label].append(score)

    # 각 생성된 텍스트에 대해 카테고리 분류 결과 추출
    for text in generated_text:
        output_replies = classifier(text, candidate_labels, multi_label=False)
        for label, score in zip(output_replies['labels'], output_replies['scores']):
            output_scores_reply[label].append(score)

    # 각각의 결과를 DataFrame으로 변환
    df_review = pd.DataFrame(output_scores_review)
    df_reply = pd.DataFrame(output_scores_reply)

    # 타겟 리뷰와 생성된 텍스트를 열로 추가
    df_review['Target Review'] = target_review
    df_reply['Generated Text'] = generated_text

    return df_review, df_reply


In [73]:
target_review = '맛은 있는데 식어서 왔어요'
generated_text = ['이용 시 불편을 참고해 더 많은 양을 제공해 주시기 바라며, 더 나은 서비스를 찾아 노력하겠습니다.','저희 음식이 짜고 느끼해서 못 드시는 분들이 종종 있는데 다음에는 더 좋은 맛을 내 주시기 위해 노력하겠습니다.','저희도 더 좋은 서비스로 보답을 할 것이니 더 열심히 활동하겠습니다','맛이없으셨군요, 죄송합니다', '식어서 제공된 음식으로 인해 아쉬운 경험을 하셨군요, 더 좋은 요리로 보답하겠습니다']

candidate_labels = ["맛", '양', "배달", "가격", '서비스']


result_review_df, result_reply_df = extract_scores_with_target_match_evaluation(target_review, generated_text, candidate_labels)

print("Review DataFrame:")
pd.DataFrame(result_review_df)

print("\nReply DataFrame:")
pd.DataFrame(result_reply_df)


/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Review DataFrame:

Reply DataFrame:


,맛,양,배달,가격,서비스,Generated Text
0,0.002211,0.559783,0.004426,0.006908,0.426673,"이용 시 불편을 참고해 더 많은 양을 제공해 주시기 바라며, 더 나은 서비스를 찾아..."
1,0.799338,0.020821,0.019571,0.020934,0.139337,저희 음식이 짜고 느끼해서 못 드시는 분들이 종종 있는데 다음에는 더 좋은 맛을 내...
2,0.019224,0.018395,0.007375,0.007314,0.947692,저희도 더 좋은 서비스로 보답을 할 것이니 더 열심히 활동하겠습니다
3,0.155924,0.430461,0.095186,0.155728,0.162701,"맛이없으셨군요, 죄송합니다"
4,0.102919,0.007602,0.029857,0.009135,0.850488,"식어서 제공된 음식으로 인해 아쉬운 경험을 하셨군요, 더 좋은 요리로 보답하겠습니다"


In [128]:
from transformers import pipeline
import pandas as pd

def extract_matching_text_with_rank(target_review, generated_text, candidate_labels):
    # Zero-shot classification 모델 불러오기
    classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", batch_size=4)

    # 결과를 저장할 변수 초기화
    label_rankings = {}  # 라벨별 확률 순위 저장
    best_matching_text = {}  # 라벨별 가장 유사한 텍스트 저장

    # 입력된 타겟 리뷰에 대한 카테고리 분류 결과 추출
    output_review = classifier(target_review, candidate_labels, multi_label=False)

    # 타겟 리뷰에 대한 라벨별 확률 순위 저장
    for label, score in zip(output_review['labels'], output_review['scores']):
        label_rankings[label] = score

    # 각 생성된 텍스트에 대해 카테고리 분류 결과 추출
    for text in generated_text:
        # 각 생성된 텍스트에 대해 카테고리 분류 결과 추출
        output_replies = classifier(text, candidate_labels, multi_label=False)

        # 라벨별 확률 순위 저장 및 가장 유사한 텍스트 저장
        for label, score in zip(output_replies['labels'], output_replies['scores']):
            if label not in label_rankings or score > label_rankings[label]:
                label_rankings[label] = score
                best_matching_text[label] = text

    # 라벨별 확률 순위를 내림차순으로 정렬
    sorted_rankings = sorted(label_rankings.items(), key=lambda x: x[1], reverse=True)

    return sorted_rankings, best_matching_text

In [129]:
target_review = '맛은 있는데 식어서 왔어요'
generated_text = ['이용 시 불편을 참고해 더 많은 양을 제공해 주시기 바라며, 더 나은 서비스를 찾아 노력하겠습니다.','저희 음식이 짜고 느끼해서 못 드시는 분들이 종종 있는데 다음에는 더 좋은 맛을 내 주시기 위해 노력하겠습니다.','저희도 더 좋은 서비스로 보답을 할 것이니 더 열심히 활동하겠습니다','맛이없으셨군요, 죄송합니다', '식어서 제공된 음식으로 인해 아쉬운 경험을 하셨군요, 더 좋은 요리로 보답하겠습니다']

candidate_labels = ["맛", '양', "배달", "가격", '서비스']


rankings, best_matching_text = extract_matching_text_with_rank(target_review, generated_text, candidate_labels)

print("라벨별 확률 순위:")
for label, score in rankings:
    print(f"{label}: {score}")

print("\n라벨별 가장 유사한 텍스트:")
for label, text in best_matching_text.items():
    print(f"{label}: {text}")



라벨별 확률 순위:
서비스: 0.9476921558380127
맛: 0.7993375062942505
양: 0.5597826838493347
배달: 0.19004814326763153
가격: 0.1557278335094452

라벨별 가장 유사한 텍스트:
양: 이용 시 불편을 참고해 더 많은 양을 제공해 주시기 바라며, 더 나은 서비스를 찾아 노력하겠습니다.
서비스: 저희도 더 좋은 서비스로 보답을 할 것이니 더 열심히 활동하겠습니다
맛: 저희 음식이 짜고 느끼해서 못 드시는 분들이 종종 있는데 다음에는 더 좋은 맛을 내 주시기 위해 노력하겠습니다.
가격: 맛이없으셨군요, 죄송합니다


In [133]:
from transformers import pipeline
import pandas as pd

def extract_matching_text_with_rank(target_review, generated_text, candidate_labels):
    # Zero-shot classification 모델 불러오기
    classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", batch_size=4)

    # 결과를 저장할 변수 초기화
    label_rankings_review = {}  # 라벨별 확률 순위 저장
    label_rankings_replies = {}  # 라벨별 확률 순위 저장

    # best_matching_text = {}  # 라벨별 가장 유사한 텍스트 저장

    # 입력된 타겟 리뷰에 대한 카테고리 분류 결과 추출
    output_review = classifier(target_review, candidate_labels, multi_label=False)

    # 타겟 리뷰에 대한 라벨별 확률 순위 저장
    for label, score in zip(output_review['labels'], output_review['scores']):
        label_rankings_review[label] = score



    # 각 생성된 텍스트에 대해 카테고리 분류 결과 추출
    for text in generated_text:
        # 각 생성된 텍스트에 대해 카테고리 분류 결과 추출
        output_replies = classifier(text, candidate_labels, multi_label=False)

        # 라벨별 확률 순위 저장 및 가장 유사한 텍스트 저장
        for label, score in zip(output_replies['labels'], output_replies['scores']):
            # if label not in label_rankings or score > label_rankings[label]:
                label_rankings_replies[label] = score
                # best_matching_text[label] = text

    # 라벨별 확률 순위를 내림차순으로 정렬
    sorted_rankings_review = sorted(label_rankings_review.items(), key=lambda x: x[1], reverse=True)
    sorted_rankings_replies = sorted(label_rankings_review.items(), key=lambda x: x[1], reverse=True)

    return sorted_rankings_review, sorted_rankings_replies

In [ ]:
target_review = '맛은 있는데 식어서 왔어요'
generated_text = ['이용 시 불편을 참고해 더 많은 양을 제공해 주시기 바라며, 더 나은 서비스를 찾아 노력하겠습니다.','저희 음식이 짜고 느끼해서 못 드시는 분들이 종종 있는데 다음에는 더 좋은 맛을 내 주시기 위해 노력하겠습니다.','저희도 더 좋은 서비스로 보답을 할 것이니 더 열심히 활동하겠습니다','맛이없으셨군요, 죄송합니다', '식어서 제공된 음식으로 인해 아쉬운 경험을 하셨군요, 더 좋은 요리로 보답하겠습니다']

candidate_labels = ["맛", '양', "배달", "가격", '서비스']

sorted_rankings, matching_text = extract_matching_text_with_rank(target_review, generated_text, candidate_labels, top_labels_count=3)

print('리뷰의 라벨별 확률:')
print(sorted_rankings)

print("라벨별 가장 유사한 텍스트:")
for label, text in matching_text.items():
    print(f"{label}: {text}")

# 텍스트 유형 확인

In [134]:
train_ft


NameError: ignored

In [131]:
from transformers import pipeline
import pandas as pd

def extract_matching_text_with_rank(target_review, generated_text, candidate_labels, top_labels_count=3):
    # Zero-shot classification 모델 불러오기
    classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", batch_size=4)

    # 결과를 저장할 변수 초기화
    label_rankings = {}  # 라벨별 확률 순위 저장
    best_matching_text = {}  # 라벨별 가장 유사한 텍스트 저장

    # 입력된 타겟 리뷰에 대한 카테고리 분류 결과 추출
    output_review = classifier(target_review, candidate_labels, multi_label=False)

    # 타겟 리뷰에 대한 라벨별 확률 순위 저장
    for label, score in zip(output_review['labels'], output_review['scores']):
        label_rankings[label] = score

    # 각 생성된 텍스트에 대해 카테고리 분류 결과 추출
    for text in generated_text:
        # 각 생성된 텍스트에 대해 카테고리 분류 결과 추출
        output_replies = classifier(text, candidate_labels, multi_label=False)

        # 라벨별 확률 순위 저장 및 가장 유사한 텍스트 저장
        for label, score in zip(output_replies['labels'], output_replies['scores']):
            if label not in label_rankings or score > label_rankings[label]:
                label_rankings[label] = score
                best_matching_text[label] = text

    # 라벨별 확률 순위를 내림차순으로 정렬
    sorted_rankings = sorted(label_rankings.items(), key=lambda x: x[1], reverse=True)

    # 상위 라벨을 선택
    top_labels = [label for label, _ in sorted_rankings[:top_labels_count]]

    # 선택한 상위 라벨과 관련된 텍스트 추출
    matching_text = {label: best_matching_text[label] for label in top_labels if label in best_matching_text}

    return sorted_rankings, matching_text


In [132]:
target_review = '맛은 있는데 식어서 왔어요'
generated_text = ['이용 시 불편을 참고해 더 많은 양을 제공해 주시기 바라며, 더 나은 서비스를 찾아 노력하겠습니다.','저희 음식이 짜고 느끼해서 못 드시는 분들이 종종 있는데 다음에는 더 좋은 맛을 내 주시기 위해 노력하겠습니다.','저희도 더 좋은 서비스로 보답을 할 것이니 더 열심히 활동하겠습니다','맛이없으셨군요, 죄송합니다', '식어서 제공된 음식으로 인해 아쉬운 경험을 하셨군요, 더 좋은 요리로 보답하겠습니다']

candidate_labels = ["맛", '양', "배달", "가격", '서비스']

sorted_rankings, matching_text = extract_matching_text_with_rank(target_review, generated_text, candidate_labels, top_labels_count=3)

print('리뷰의 라벨별 확률:')
print(sorted_rankings)

print("라벨별 가장 유사한 텍스트:")
for label, text in matching_text.items():
    print(f"{label}: {text}")


/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


리뷰의 라벨별 확률:
[('서비스', 0.9476921558380127), ('맛', 0.7993375062942505), ('양', 0.5597826838493347), ('배달', 0.19004814326763153), ('가격', 0.1557278335094452)]
라벨별 가장 유사한 텍스트:
서비스: 저희도 더 좋은 서비스로 보답을 할 것이니 더 열심히 활동하겠습니다
맛: 저희 음식이 짜고 느끼해서 못 드시는 분들이 종종 있는데 다음에는 더 좋은 맛을 내 주시기 위해 노력하겠습니다.
양: 이용 시 불편을 참고해 더 많은 양을 제공해 주시기 바라며, 더 나은 서비스를 찾아 노력하겠습니다.


In [122]:
from transformers import pipeline
import pandas as pd

def extract_matching_text_with_rank(target_review, generated_text, candidate_labels):
    # Zero-shot classification 모델 불러오기
    classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", batch_size=4)

    # 결과를 저장할 변수 초기화
    label_rankings = {}  # 라벨별 확률 순위 저장

    # 입력된 타겟 리뷰에 대한 카테고리 분류 결과 추출
    output_review = classifier(target_review, candidate_labels, multi_label=False)

    # 타겟 리뷰에 대한 라벨별 확률 순위 저장
    for i, (label, score) in enumerate(zip(output_review['labels'], output_review['scores'])):
        label_rankings[label] = {'rank': i + 1, 'score': score}

    # 각 생성된 텍스트에 대해 카테고리 분류 결과 추출
    text_matching_info = {label: {'text': None, 'rank': None, 'score': None} for label in candidate_labels}
    for text in generated_text:
        # 각 생성된 텍스트에 대해 카테고리 분류 결과 추출
        output_replies = classifier(text, candidate_labels, multi_label=False)

        # 라벨별 확률 순위 저장
        for i, (label, score) in enumerate(zip(output_replies['labels'], output_replies['scores'])):
            if label in text_matching_info:
                if text_matching_info[label]['rank'] is None or i < text_matching_info[label]['rank']:
                    text_matching_info[label]['text'] = text
                    text_matching_info[label]['rank'] = i
                    text_matching_info[label]['score'] = score

    # 타겟 리뷰의 상위 3개 라벨과 일치하는 텍스트 추출
    matching_text = {label: text_matching_info[label] for label, rank_info in label_rankings.items() if rank_info['rank'] <= 3}

    return matching_text


In [89]:
# 사용 예시
target_review = '맛은 있는데 식어서 왔어요'
generated_text = ['이용 시 불편을 참고해 더 많은 양을 제공해 주시기 바라며, 더 나은 서비스를 찾아 노력하겠습니다.','저희 음식이 짜고 느끼해서 못 드시는 분들이 종종 있는데 다음에는 더 좋은 맛을 내 주시기 위해 노력하겠습니다.','저희도 더 좋은 서비스로 보답을 할 것이니 더 열심히 활동하겠습니다','맛이없으셨군요, 죄송합니다', '식어서 제공된 음식으로 인해 아쉬운 경험을 하셨군요, 더 좋은 요리로 보답하겠습니다']

candidate_labels = ["맛", '양', "배달", "가격", '서비스']

matching_text = extract_matching_text_with_rank(target_review, generated_text, candidate_labels)

print("라벨별 가장 유사한 텍스트:\n")
for label, text_info in matching_text.items():
    print(f"라벨: {label}")
    print(f"텍스트: {text_info['text']}")
    print(f"순위: {text_info['rank']+1}")
    print(f"점수: {text_info['score']}")

라벨별 가장 유사한 텍스트:

라벨: 맛
텍스트: 저희 음식이 짜고 느끼해서 못 드시는 분들이 종종 있는데 다음에는 더 좋은 맛을 내 주시기 위해 노력하겠습니다.
순위: 1
점수: 0.7993375062942505
라벨: 배달
텍스트: 식어서 제공된 음식으로 인해 아쉬운 경험을 하셨군요, 더 좋은 요리로 보답하겠습니다
순위: 3
점수: 0.029856927692890167
라벨: 서비스
텍스트: 저희도 더 좋은 서비스로 보답을 할 것이니 더 열심히 활동하겠습니다
순위: 1
점수: 0.9476921558380127


# 유사도
- 유사도 계산은 1:1로 계산할 수밖에 없음

In [111]:

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import jaccard_score
bert_model = SentenceTransformer('jhgan/ko-sroberta-multitask')

def evaluate_similarity(input_text, generated_text, alpha=0.9): # cosine 가중치 조절
    input_embedding = bert_model.encode(input_text)
    generated_embedding = bert_model.encode(generated_text)

    # 코사인 유사도
    cosine_sim = 1 - pairwise_distances([input_embedding], [generated_embedding], metric='cosine')[0][0]

    # 자카드 유사도
    input_tokens = set(input_text.split())
    generated_tokens = set(generated_text.split())
    jaccard_sim = len(input_tokens.intersection(generated_tokens)) / len(input_tokens.union(generated_tokens))
    # jaccard_sim = jaccard_score(input_tokens, generated_tokens)

    # 가중 평균 내보기
    weighted_sim = alpha*cosine_sim + (1 - alpha)*jaccard_sim

    return weighted_sim

In [126]:
input_text = '맛은 있는데 식어서 왔어요\n'
generated_text = ['이용 시 불편을 참고해 더 많은 양을 제공해 주시기 바라며, 더 나은 서비스를 찾아 노력하겠습니다.','저희 음식이 짜고 느끼해서 못 드시는 분들이 종종 있는데 다음에는 더 좋은 맛을 내 주시기 위해 노력하겠습니다.','저희도 더 좋은 서비스로 보답을 할 것이니 더 열심히 활동하겠습니다','맛이없으셨군요, 죄송합니다', '식어서 제공된 음식으로 인해 아쉬운 경험을 하셨군요, 더 좋은 요리로 보답하겠습니다']

candidate_labels = ["맛", '양', "배달", "가격", '서비스']

evaluate_similarity("맛은 있는데 양이 적어요",'맛있게 드셔주셔서 감사합니다. 다음에 양 많이 드릴게요')

# sorted_text_similarity = evaluate_similarity(input_text, generated_text)
# print("입력 텍스트:", input_text)
# print("텍스트 유사도 (점수 높은 순으로 정렬):")

# for text, similarity in sorted_text_similarity:
#     print(f"점수: {similarity}, 텍스트: {text}")

print("input text:", input_text,'\n')

print("유사도 순위")
score_list = []  # 유사도를 저장할 리스트 초기화
for i, text in enumerate(generated_text):
    similarity = evaluate_similarity(input_text, generated_text[i])
    score_list.append((similarity, (input_text, generated_text[i]), text))

# 유사도 기준으로 정렬
score_list.sort(key=lambda x: x[0], reverse=True)

# 정렬된 결과 출력

for rank, (similarity, (input_text, generated_text_i), text) in enumerate(score_list):
    print(f"순위 {rank + 1}, 유사도 점수: {similarity} {generated_text_i}")
    # print(f"생성된 텍스트: {generated_text_i}\n")


print('라벨 확률 추출')


input text: 맛은 있는데 식어서 왔어요
 

유사도 순위
순위 1, 유사도 점수: 0.6229709659303938 식어서 제공된 음식으로 인해 아쉬운 경험을 하셨군요, 더 좋은 요리로 보답하겠습니다
순위 2, 유사도 점수: 0.4829598426818848 맛이없으셨군요, 죄송합니다
순위 3, 유사도 점수: 0.4472827482223511 저희 음식이 짜고 느끼해서 못 드시는 분들이 종종 있는데 다음에는 더 좋은 맛을 내 주시기 위해 노력하겠습니다.
순위 4, 유사도 점수: 0.07318026423454285 이용 시 불편을 참고해 더 많은 양을 제공해 주시기 바라며, 더 나은 서비스를 찾아 노력하겠습니다.
순위 5, 유사도 점수: 0.021308916807174682 저희도 더 좋은 서비스로 보답을 할 것이니 더 열심히 활동하겠습니다
라벨 확률 추출


# 라벨확률 추출 + 유사도

In [125]:
# 각 생성된 텍스트에 대한 유사도 점수 계산
similarity_scores = [evaluate_similarity(input_text, text) for text in generated_text]

# 각 생성된 텍스트에 대한 라벨 추출 정보 계산
label_matching_info = extract_matching_text_with_rank(input_text, generated_text, candidate_labels)

# 최종 종합 점수 계산 및 최적 텍스트 선택
best_text = None
best_score = -1  # 초기 최저 점수 설정

for i, text in enumerate(generated_text):
    similarity_score = similarity_scores[i]

    if i + 1 in label_matching_info:
        label_score = label_matching_info[i + 1]['score']
    else:
        label_score = 0  # 라벨 스코어가 없을 경우 0으로 설정

    # 종합 점수 계산 (유사도 점수와 라벨 스코어 고려)
    combined_score = similarity_score * label_score

    # 최적 텍스트 업데이트
    if combined_score > best_score:
        best_text = text
        best_score = combined_score

# 최종 답변 출력
print("최종 답변:")
print(best_text)


/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


최종 답변:
이용 시 불편을 참고해 더 많은 양을 제공해 주시기 바라며, 더 나은 서비스를 찾아 노력하겠습니다.


In [44]:
def chatbot_with_evaluation(model, tokenizer, max_len, device, num_samples=5):
    model.eval()
    while True:
        text = input("user > ").strip()
        if text == "quit":
            break

        text = "<q>" + text + "</s><a>"
        x = tokenizer.encode(text, return_tensors="pt").to(device)
        q_len = len(text) + 1

        best_generated_text = None
        best_similarity_score = -1.0
        generated_texts = []  # 답변 후보군을 담을 리스트
        for i in range(num_samples):
            result_ids = model.generate(x,
                                        max_length=max_len,
                                        repetition_penalty=2.0,
                                        use_cache=True,
                                        do_sample=True,
                                        temperature=0.9,
                                        top_k=50,
                                        num_return_sequences=1
                                        )
            generated_text = tokenizer.decode(result_ids[0])
            generated_text = generated_text[q_len:-4]
            generated_text = paraphrase_text(generated_text)

            similarity_score = evaluate_similarity(text, generated_text)
            generated_texts.append((similarity_score,generated_text))

            candidate_labels = ["맛", '양', "배달", "가격", '서비스']
            extract_scores(text, candidate_labels)
            extract_scores(generated_texts, candidate_labels)

            print(generated_texts[i])

            if similarity_score > best_similarity_score:
                best_similarity_score = similarity_score
                best_generated_text = generated_text

        print()
        print("Best_reply >", best_generated_text)
        print("Best_similarity_score:", best_similarity_score)
        print("--------------------------------------------------------------------------------------------")


In [1]:
chatbot_with_evaluation(loaded_model,tokenizer,128,device)

NameError: ignored


- 0914_trainsentiment_0913+gpt

In [42]:
chatbot(loaded_model,tokenizer,128,device)

NameError: ignored

- 0905_kogpt2_chat

In [ ]:
chatbot(loaded_model,tokenizer,128,device)

user > 맛이 없어요
bot > 고객님 ~! 먼저 죄송합니다. 부족한 부분으로 실망시켜  사과드리며 늘 저희를 사랑해 주시는 고객님이시군요 정성스러운 사진 리뷰와도 너무 감사한데, 먹음직스럽게 찍어주신 소중한 말씀까지.
--------------------------------------------------------------------------------------------
user > 맛은 있는데 배달이 식어서 왔어요.
bot > 소중한 리뷰 감사드립니다 저희 매장은 맛과 양, 서비스 등 모든 면에서 최고가 되기 위해 항상 최선을 다하고 있습니다, 언제든지 부족함 없이 드실 수 있도록 준비해 보내드리겠습니다.! 앞으로도 만족만 드릴 만반의 준비를 하여 계속 마음속에 남아 있고 싶거든요 <s>~ 좋은 하루 되세요. 고객님 ~
--------------------------------------------------------------------------------------------
user > 먹다가 이물질이 나왔어요ㅠㅠ
bot > 오늘 하루를 시작하며 고객님의 리뷰을 보고 웃음 짓게 됩니다 다음에도 점심 식사를 기분 좋게 드실 수 있길 바랍니다. 저희는 항상 맛있는 음식 제공에 힘내며 언제라도 생각나는 맛집이 되도록 노력하겠습니다, 앞으로도 더욱 만족스러운 식사하실 땐 더 큰 기쁨으로 인사드리겠습니다.
--------------------------------------------------------------------------------------------
user > 가격에 비해 양이 너무 적어서 아쉽네요,,,
bot > 맛있게 드셨다니 다행입니다! 찾아주시는 피드백은 언제든 잘 수렴하고 이를 통해 나날이 발전하는 곳이 되도록 하겠습니다, 더욱더 좋은 모습으로 찾아가겠습니다.<s>. 기쁨 가득한 하루 보내세요 <s>, 감사해요.<s><s><s><s><s> 소중한 리뷰에 제가 느낀 이 행복만큼 고객님도 항상 노력하시길 바라며 오늘

- 0906_negative_kogpt2_chat

In [ ]:
chatbot(model,tokenizer,128,device)

user > 맛이 없어요
bot > 안녕하세요! 저희 맛집이 만족한 식사시간이 되지 못해서 죄송합니다.. 양이 부족하다.?
--------------------------------------------------------------------------------------------
user > 맛은 있는데 배달이 식어서 왔어요.
bot > 고객님 식사에 불편을 드려 너무 죄송합니다 다음엔 더욱 신경 써서 만족하실 수 있도록 전달하겠습니다, 감사합니다.<s> <unk>얌꿍이 약간 매콤한 맛으로 바뀌어서요~ 더 맛있는 요리로 찾아뵐게요. 가성비 있고 푸짐한 양 담아 제공하는 팔당이 되겠습니다.<s> <b>이상, 당일 납품받는 재료로 최고의 신선도를 유지하여 드리는 회사가 되도록 노력할 것이 무엇보다 중요한데도 불구하고 저희 음식 포장 문제가 발생을 하니 정말이지에서 누락되는 경우가 발생되고 있네요, 다시 한
--------------------------------------------------------------------------------------------
user > 먹다가 이물질이 나왔어요ㅠㅠ
bot > 고객님들께 만족감을 드리려고 연구하고 노력하고 있는데 아직 많이 부족한 것 같습니다 시간 내시어나 말씀해 주시면 감사하겠습니다.<s> <blight difference in size of the nice wit must have big chance, I recommend you try little phortfyetangnam. HMT 4. 콜라 1.
--------------------------------------------------------------------------------------------
user > 가격에 비해 양이 너무 적어서 아쉽네요,,,
bot >  양적으로 실망하셨다니 너무 죄송합니다. 다음번엔 더욱 맛있는 음식으로 보답 드리도록 하겠습니다.<s> 감사합니다! <s> 사진은 다시 한번 더 수정해서 올리거나

- 0906_positive_kogpt2_chat

In [ ]:
chatbot(pos_model,tokenizer,128,device)

user > 맛이 없어요
bot >  안녕하세요! 푸짐하고 맛있는 헬로 팬입니다 맛있게 잘 드셔주신 것 같아 기분 좋네요. 앞으로도 변함없는 모습으로 보답하는헬로, 닭갈비 생각나는 날 되겠습니다, 다음에 또 만나 <unk> 주세요 
--------------------------------------------------------------------------------------------
user > 맛은 있는데 배달이 식어서 왔어요.
bot > 안녕하세요! 소중한 별점 만 점 리뷰 감사합니다 즐거운 식사가 되신 거 같아 너무 뿌듯해요, 행복하게 보내셔서 진심으로 감동의 물결이 휘몰아치네요. 기분 좋은 칭찬 사진 너무나도 이쁘게 찍어주셨는데 맛있게 드시고 예쁜 포토리뷰까지 남겨 주시니 그저 아쉽지 않습니다, 이렇게 받은 신뢰에 꼭 보답할 수 있도록 매일 신선한 재료로 정성을 다하겠습니다.
--------------------------------------------------------------------------------------------
user > 먹다가 이물질이 나왔어요ㅠㅠ
bot > 안녕하세요. 이렇게 저희에게 귀하고 소중한 평가 남겨주셔서 너무 감사해 앞으로도 신선한 재료 아끼지 않고 정성을 가득 담아 준비 약속드릴게요, 무더운 날씨 항상 더위 조심하시고 늘 행복만 전해드리겠습니다! 
--------------------------------------------------------------------------------------------
user > 가격에 비해 양이 너무 적어서 아쉽네요,,,
bot > 안녕하세요~ 소중한 시간 내셔서 리뷰까지 감동입니다! 고객님께서 남겨주신 별점에, 저희를 춤추게 합니, 기분이 좋습니다, 항상 힘내어 드실 수 있도록 꾸준히 노력하겠습니다. 앞으로도 행복한 하루 보내시길 바랄 테니 많이 애용해 주시고, 감기 조심하시고 파이팅 하오 
-------------------------------